In [36]:
%matplotlib inline
import numpy as np
from alphamind.api import *
from PyFin.api import *

engine = SqlEngine()

In [37]:
start_date = '2012-01-01'
end_date = '2017-12-31'

features = ['roe_q',
            'ep_q',
            'DivP',
            'cfinc1_q',
            'EBIT',
            'EARNYILD',
            'EPIBS']

freq = '5b'
batch = 8
universe = Universe('custom', ['zz500'])
benchmark = 905
neutralized_risk = ['SIZE'] + industry_styles
horizon = map_freq(freq)

In [38]:
factor_data = fetch_data_package(engine,
                                 features,
                                 start_date,
                                 end_date,
                                 '5b',
                                 universe,
                                 benchmark,
                                 batch=batch,
                                 warm_start=batch,
                                 neutralized_risk=neutralized_risk,
                                 pre_process=[winsorize_normal, standardize],
                                 post_process=[winsorize_normal, standardize])

2018-01-04 17:53:36,534 - ALPHA_MIND - INFO - Starting data package fetching ...
2018-01-04 17:55:46,755 - ALPHA_MIND - INFO - Loading data is finished
2018-01-04 17:55:47,900 - ALPHA_MIND - INFO - Data processing is finished


In [39]:
index_data = engine.fetch_dx_return_index_range

In [40]:
features = factor_data['x_names']

train_x = factor_data['train']['x']
train_y = factor_data['train']['y']
train_risk = factor_data['train']['risk']
ref_dates = sorted(train_x.keys())

predict_x = factor_data['predict']['x']
predict_y = factor_data['predict']['y']
predict_risk = factor_data['predict']['risk']
settlement = factor_data['settlement']

## Old Model
------------------

In [56]:
train_dates = list(train_x.keys())
old_train_scores = []
old_predict_scores = []

for date in train_dates:
    print(date)
    x = train_x[date]
    y = train_y[date]
    
    #model = LassoRegression(alpha=0.002, fit_intercept=False, features=features)
    model = RandomForestRegressor(n_estimators=1000, max_features='sqrt', max_depth=3)
    #model = LinearRegression(fit_intercept=False, features=features)
    model.fit(x, y)
    old_train_scores.append(model.score(x, y))
    
    p_x = predict_x[date]
    p_y = predict_y[date]
    old_predict_scores.append(model.score(p_x, p_y))

2012-02-08 00:00:00


D:\ProgramData\IntelPython3_2018\lib\site-packages\alpha_mind-0.1.0-py3.6-win-amd64.egg\alphamind\model\treemodel.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.impl.fit(x, y)


2012-02-15 00:00:00


D:\ProgramData\IntelPython3_2018\lib\site-packages\alpha_mind-0.1.0-py3.6-win-amd64.egg\alphamind\model\treemodel.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.impl.fit(x, y)


2012-02-22 00:00:00


...

D:\ProgramData\IntelPython3_2018\lib\site-packages\alpha_mind-0.1.0-py3.6-win-amd64.egg\alphamind\model\treemodel.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.impl.fit(x, y)


2017-12-20 00:00:00


D:\ProgramData\IntelPython3_2018\lib\site-packages\alpha_mind-0.1.0-py3.6-win-amd64.egg\alphamind\model\treemodel.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.impl.fit(x, y)


2017-12-27 00:00:00


D:\ProgramData\IntelPython3_2018\lib\site-packages\alpha_mind-0.1.0-py3.6-win-amd64.egg\alphamind\model\treemodel.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.impl.fit(x, y)


In [57]:
print(np.mean(old_train_scores))
print(np.mean(old_predict_scores))

0.0284819052241
-0.504361905951


## New Model
---------------

In [43]:
def cross_product(x, y):
    n, m = x.shape
    res = []
    
    for j in range(m):
        res.append(x[:, [j]] * y)
        
    return np.concatenate(res, axis=1)

In [54]:
train_dates = list(train_x.keys())
new_train_scores = []
new_predict_scores = []

for date in train_dates:
    print(date)
    x = train_x[date]
    y = train_y[date]
    risk = train_risk[date][:, 1:]
    new_x = cross_product(x, risk)
    
    #model = LassoRegression(alpha=0.002, fit_intercept=False, features=[])
    model = RandomForestRegressor(n_estimators=1000, max_features='sqrt', max_depth=3)
    model.fit(new_x, y)
    new_train_scores.append(model.score(new_x, y))
    
    p_x = predict_x[date]
    p_y = predict_y[date]
    p_risk = predict_risk[date][:, 1:]
    new_p_x = cross_product(p_x, p_risk)
    new_predict_scores.append(model.score(new_p_x, p_y))

2012-02-08 00:00:00


D:\ProgramData\IntelPython3_2018\lib\site-packages\alpha_mind-0.1.0-py3.6-win-amd64.egg\alphamind\model\treemodel.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.impl.fit(x, y)


2012-02-15 00:00:00


D:\ProgramData\IntelPython3_2018\lib\site-packages\alpha_mind-0.1.0-py3.6-win-amd64.egg\alphamind\model\treemodel.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.impl.fit(x, y)


2012-02-22 00:00:00


...

D:\ProgramData\IntelPython3_2018\lib\site-packages\alpha_mind-0.1.0-py3.6-win-amd64.egg\alphamind\model\treemodel.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.impl.fit(x, y)


2017-12-20 00:00:00


D:\ProgramData\IntelPython3_2018\lib\site-packages\alpha_mind-0.1.0-py3.6-win-amd64.egg\alphamind\model\treemodel.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.impl.fit(x, y)


2017-12-27 00:00:00


D:\ProgramData\IntelPython3_2018\lib\site-packages\alpha_mind-0.1.0-py3.6-win-amd64.egg\alphamind\model\treemodel.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.impl.fit(x, y)


In [55]:
print(np.mean(new_train_scores))
print(np.mean(new_predict_scores))

0.0197222525364
-0.502156400496
